In [23]:
!pip install langchain

In [24]:
!pip install langchain-openai

In [25]:
!pip install tiktoken

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = 'OPEN_API_KEY' # 발급받은 자신의 OpenAI API Key를 넣어줍니다.

In [27]:
from langchain_openai import ChatOpenAI

# Model
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125")

# chain 실행
result = llm.invoke("지구의 자전주기가 어떻게 돼?")

In [28]:
result.content

'지구의 자전주기는 지구의 자전축의 회전 속도와 관련이 있습니다. \n\n지구의 자전축은 지구의 중심을 통과하는 상상상의 직선인데, 이 자전축을 중심으로 지구가 하루에 한 바퀴를 도는데 약 24시간이 걸립니다. 이렇게 지구가 자전축을 중심으로 도는 것을 자전이라고 합니다. \n\n자전은 지구의 둥근 모양 때문에 일부 지역은 태양의 빛을 받게 되어 낮시간이 생기고, 일부 지역은 반대편에 있어서 밤이 됩니다. 이렇게 자전이 지구의 24시간 주기로 일어나기 때문에 우리는 낮과 밤이 반복됩니다.'

In [29]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# prompt + model + output parser
prompt = ChatPromptTemplate.from_template("You are an expert in astronomy. Answer the question. <Question>: {input}")
llm = ChatOpenAI(model="gpt-4o-mini")
output_parser = StrOutputParser() # StrOutputParser는 출력값에서 문자열만 추출

# LCEL chaining
chain = prompt | llm | output_parser

# chain 호출
chain.invoke({"input": "지구의 자전 주기는?"}) # 여기서 invoke()는 체인을 실행하는 메서드

'지구의 자전 주기는 약 24시간입니다. 정확히 말하면, 평균적으로 23시간 56분 4초, 즉 약 24시간이 소요되어 한 번 자전합니다. 이를 “태양일”이라고 하며, 하루의 기초가 되는 시간 단위입니다.'

In [30]:
prompt1 = ChatPromptTemplate.from_template("translates {korean_word} to English.")
prompt2 = ChatPromptTemplate.from_template(
    "explain {english_word} using oxford dictionary to me in Korean."
)

llm = ChatOpenAI(model="gpt-4o-mini")

chain1 = prompt1 | llm | StrOutputParser()

chain1.invoke({"korean_word":"미래"})

'The Korean word "미래" translates to "future" in English.'

In [31]:
chain2 = (
    {"english_word": chain1}
    | prompt2
    | llm
    | StrOutputParser()
)

chain2.invoke({"korean_word":"미래"})

'"미래"라는 한국어 단어는 영어로 "future"로 번역됩니다. 옥스퍼드 사전에서는 "future"를 다음과 같이 정의하고 있습니다:\n\n1. **명사**: 아직 오지 않은 시간 또는 사건들, 특히 현재 시점 이후의 시간.\n2. **형용사**: 앞으로 일어날 것으로 예상되거나 계획된 것과 관련된.\n\n즉, "미래"는 현재의 시점에서 앞으로 있을 일이나 시간에 대한 개념을 나타내는 단어입니다.'

In [32]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

# 1. 컴포넌트 정의
prompt = ChatPromptTemplate.from_template("지구과학에서 {topic}에 대해 간단히 설명해주세요.")
model = ChatOpenAI(model="gpt-4o-mini")
output_parser = StrOutputParser()

# 2. 체인 생성
chain = prompt | model | output_parser

# 3. invoke 메소드 사용
result = chain.invoke({"topic": "지구 자전"})
print("invoke 결과:", result)

invoke 결과: 지구 자전은 지구가 자신의 축을 중심으로 하루에 한 번 회전하는 과정을 말합니다. 이 자전은 약 24시간이 걸리며, 이를 기준으로 하루가 정의됩니다. 지구의 자전은 동쪽에서 서쪽으로 이루어지며, 이로 인해 태양이 동쪽에서 떠오르고 서쪽으로 지는 모습을 관찰할 수 있습니다.

자전의 결과로 생기는 주요 현상 중 하나는 낮과 밤의 발생입니다. 또한, 자전 속도는 지구의 형태에도 영향을 미쳐 적도 부분이 약간 부풀어 오른 편평한 구형을 띠게 됩니다. 이러한 자전은 지구의 기후와 날씨 패턴에도 영향을 미치며, 지구의 자전축은 약 23.5도 기울어져 있어 계절의 변화에도 중요한 역할을 합니다.


In [33]:
# batch 메소드 사용
topics = ["지구 공전", "화산 활동", "대륙 이동"]
results = chain.batch([{"topic": t} for t in topics])
for topic, result in zip(topics, results):
    print(f"{topic} 설명: {result[:50]}...")  # 결과의 처음 50자만 출력

지구 공전 설명: 지구 공전은 지구가 태양을 중심으로 일정한 궤도를 따라 돌면서 이동하는 과정을 말합니다. ...
화산 활동 설명: 화산 활동은 지구 내부의 마그마가 지각을 뚫고 나와 지표면으로 분출되는 과정을 말합니다. ...
대륙 이동 설명: 대륙 이동 이론은 지구의 대륙들이 과거에 하나의 대륙(팬게아)으로 존재했으며, 시간이 지나...


In [34]:
# stream 메소드 사용
stream = chain.stream({"topic": "지진"})
print("stream 결과:")
for chunk in stream:
    print(chunk, end="", flush=True)
print()

stream 결과:
지진은 지구의 지각에서 발생하는 자연 현상으로, 주로 지각 내부의 판이 서로 충돌하거나 이동할 때 발생합니다. 이러한 과정에서 에너지가 축적되어 갑작스럽게 방Released됨에 따라 지구의 진동이 발생하게 됩니다. 이 진동은 지표면에 도달하여 사람과 건물에 영향을 미치고, 때때로 큰 피해를 초래할 수 있습니다.

지진의 주요 원인은 다음과 같습니다:

1. **판 운동**: 지구의 외층인 지각은 여러 개의 판으로 나뉘어 있으며, 이 판들이 서로 만나거나 멀어지는 과정에서 지진이 발생합니다.
2. **단층**: 지각이 갈라져 있는 부분인 단층에서 힘이 축적되고, 그 힘이 갑작스럽게 해소될 때 지진이 발생합니다.
3. **화산 활동**: 마그마가 지표로 상승하면서 불안정한 상태가 될 때도 지진이 발생할 수 있습니다.

지진의 강도는 리히터 규모로 측정되며, 이를 통해 지진의 파괴력과 영향을 평가할 수 있습니다. 지진 후에는 여진이 발생할 수 있으며, 이는 주 지진 이후에 발생하는 추가적인 진동입니다. 지진 예측은 어렵지만, 지진 발생 지역에서는 예방과 대비가 중요합니다.


In [35]:
import nest_asyncio
import asyncio

# nest_asyncio 적용 (구글 코랩 등 주피터 노트북에서 실행 필요)
nest_asyncio.apply()

# 비동기 메소드 사용 (async/await 구문 필요)
async def run_async():
    result = await chain.ainvoke({"topic": "해류"})
    print("ainvoke 결과:", result[:50], "...")

asyncio.run(run_async())

ainvoke 결과: 해류는 바다에서 물이 일정한 방향으로 흐르는 현상을 말합니다. 해류는 여러 요인에 의해 형 ...
